# Bab 7: Membangun Aplikasi Chat
## OpenAI API Quickstart

Notebook ini diadaptasi dari [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) yang berisi notebook untuk mengakses layanan [Azure OpenAI](notebook-azure-openai.ipynb).

API OpenAI Python juga dapat digunakan dengan Model Azure OpenAI, dengan beberapa penyesuaian. Pelajari lebih lanjut tentang perbedaannya di sini: [Cara beralih antara endpoint OpenAI dan Azure OpenAI dengan Python](https://learn.microsoft.com/azure/ai-services/openai/how-to/switching-endpoints?WT.mc_id=academic-109527-jasmineg)


# Gambaran Umum  
"Model bahasa besar adalah fungsi yang memetakan teks ke teks. Diberikan sebuah input berupa string teks, model bahasa besar akan mencoba memprediksi teks apa yang akan muncul berikutnya"(1). Notebook "quickstart" ini akan memperkenalkan konsep LLM tingkat tinggi, persyaratan inti paket untuk memulai dengan AML, pengenalan ringan tentang desain prompt, serta beberapa contoh singkat dari berbagai kasus penggunaan.


## Daftar Isi  

[Gambaran Umum](../../../../07-building-chat-applications/python)  
[Cara Menggunakan Layanan OpenAI](../../../../07-building-chat-applications/python)  
[1. Membuat Layanan OpenAI Anda](../../../../07-building-chat-applications/python)  
[2. Instalasi](../../../../07-building-chat-applications/python)    
[3. Kredensial](../../../../07-building-chat-applications/python)  

[Studi Kasus](../../../../07-building-chat-applications/python)    
[1. Merangkum Teks](../../../../07-building-chat-applications/python)  
[2. Mengklasifikasikan Teks](../../../../07-building-chat-applications/python)  
[3. Membuat Nama Produk Baru](../../../../07-building-chat-applications/python)  
[4. Melatih Ulang Klasifikasi](../../../../07-building-chat-applications/python)  

[Referensi](../../../../07-building-chat-applications/python)


### Bangun prompt pertamamu  
Latihan singkat ini akan memberikan pengenalan dasar tentang cara mengirimkan prompt ke model OpenAI untuk tugas sederhana "merangkum".

**Langkah-langkah**:  
1. Instal library OpenAI di lingkungan python-mu  
2. Muat library pembantu standar dan atur kredensial keamanan OpenAI yang biasa kamu gunakan untuk OpenAI Service yang sudah kamu buat  
3. Pilih model untuk tugasmu  
4. Buat prompt sederhana untuk model  
5. Kirim permintaanmu ke API model!


In [ ]:
%pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )


### 3. Menemukan model yang tepat  
Model GPT-3.5-turbo atau GPT-4 dapat memahami dan menghasilkan bahasa alami.


In [ ]:
# Select the General Purpose curie model for text
model = "gpt-3.5-turbo"

## 4. Desain Prompt  

"Keajaiban dari model bahasa besar adalah dengan dilatih untuk meminimalkan kesalahan prediksi pada sejumlah besar teks, model akhirnya mempelajari konsep-konsep yang berguna untuk prediksi tersebut. Misalnya, mereka mempelajari konsep seperti"(1):

* cara mengeja
* cara kerja tata bahasa
* cara memparafrasekan
* cara menjawab pertanyaan
* cara melakukan percakapan
* cara menulis dalam banyak bahasa
* cara membuat kode
* dan lain-lain

#### Cara mengendalikan model bahasa besar  
"Dari semua masukan ke model bahasa besar, yang paling berpengaruh adalah prompt teks(1).

Model bahasa besar dapat diprompt untuk menghasilkan output dengan beberapa cara:

Instruksi: Beritahu model apa yang kamu inginkan  
Penyelesaian: Buat model melanjutkan awal dari apa yang kamu inginkan  
Demonstrasi: Tunjukkan pada model apa yang kamu inginkan, dengan:  
Beberapa contoh di dalam prompt  
Ratusan atau ribuan contoh dalam dataset pelatihan fine-tuning"



#### Ada tiga pedoman dasar dalam membuat prompt:

**Tunjukkan dan jelaskan**. Buat jelas apa yang kamu inginkan, baik melalui instruksi, contoh, atau kombinasi keduanya. Jika kamu ingin model mengurutkan daftar item secara alfabetis atau mengklasifikasikan paragraf berdasarkan sentimen, tunjukkan bahwa itulah yang kamu inginkan.

**Berikan data yang berkualitas**. Jika kamu ingin membangun sebuah pengklasifikasi atau membuat model mengikuti pola tertentu, pastikan ada cukup contoh. Pastikan juga untuk memeriksa kembali contoh-contohmu — model biasanya cukup pintar untuk memahami kesalahan ejaan dasar dan tetap memberikan respons, tapi model juga bisa menganggap itu disengaja dan hal ini bisa memengaruhi responsnya.

**Periksa pengaturanmu.** Pengaturan temperature dan top_p mengontrol seberapa deterministik model dalam menghasilkan respons. Jika kamu meminta respons yang hanya punya satu jawaban benar, sebaiknya atur nilai ini lebih rendah. Jika kamu ingin respons yang lebih beragam, kamu bisa menaikkannya. Kesalahan paling umum yang dilakukan orang dengan pengaturan ini adalah menganggapnya sebagai pengatur "kepintaran" atau "kreativitas".


Sumber: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Merangkum Teks  
#### Tantangan  
Rangkum teks dengan menambahkan 'tl;dr:' di akhir sebuah paragraf. Perhatikan bagaimana model dapat melakukan berbagai tugas tanpa instruksi tambahan. Kamu bisa bereksperimen dengan prompt yang lebih deskriptif daripada tl;dr untuk mengubah perilaku model dan menyesuaikan ringkasan yang kamu terima(3).  

Penelitian terbaru menunjukkan peningkatan signifikan pada banyak tugas dan tolok ukur NLP dengan melakukan pre-training pada korpus teks besar, lalu fine-tuning pada tugas tertentu. Meskipun arsitekturnya biasanya tidak bergantung pada tugas, metode ini tetap membutuhkan dataset fine-tuning khusus tugas yang terdiri dari ribuan hingga puluhan ribu contoh. Sebaliknya, manusia umumnya bisa melakukan tugas bahasa baru hanya dari beberapa contoh atau instruksi sederhana—sesuatu yang masih sulit dicapai oleh sistem NLP saat ini. Di sini kami menunjukkan bahwa memperbesar skala model bahasa secara signifikan meningkatkan performa few-shot yang tidak bergantung pada tugas, bahkan kadang bisa menyaingi pendekatan fine-tuning terbaik sebelumnya.



Tl;dr


# Latihan untuk beberapa kasus penggunaan  
1. Merangkum Teks  
2. Mengklasifikasikan Teks  
3. Membuat Nama Produk Baru


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Klasifikasikan Teks  
#### Tantangan  
Klasifikasikan item ke dalam kategori yang diberikan saat inferensi. Pada contoh berikut, kita memberikan baik kategori maupun teks yang akan diklasifikasikan di prompt (*playground_reference).

Pertanyaan Pelanggan: Halo, salah satu tombol di keyboard laptop saya baru saja rusak dan saya butuh pengganti:

Kategori yang diklasifikasikan:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Buat Nama Produk Baru
#### Tantangan
Buat nama produk dari kata-kata contoh. Di sini kami sertakan informasi tentang produk yang akan dibuatkan namanya dalam prompt. Kami juga memberikan contoh serupa untuk menunjukkan pola yang diinginkan. Nilai temperatur juga kami atur tinggi agar hasilnya lebih acak dan inovatif.

Deskripsi produk: Mesin pembuat milkshake rumahan
Kata kunci: cepat, sehat, ringkas.
Nama produk: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Deskripsi produk: Sepasang sepatu yang bisa muat di semua ukuran kaki.
Kata kunci: adaptif, pas, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.chat.completions.create(
  model=model,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Referensi  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [Contoh OpenAI Studio](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Praktik terbaik untuk fine-tuning GPT-3 dalam mengklasifikasikan teks](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Untuk Bantuan Lebih Lanjut  
[Tim Komersialisasi OpenAI](AzureOpenAITeam@microsoft.com)


# Kontributor
* Louis Li



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan terjemahan yang akurat, harap diketahui bahwa terjemahan otomatis dapat mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa penerjemah profesional. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang timbul akibat penggunaan terjemahan ini.
